## いろんな方法を使ってハイパーパラメータの調整をしてみる(特にGA)
参考 : https://qiita.com/cvusk/items/1f3b178f34c39beb29ff
### 目的 : Python版のGAの使い方と機械学習におけるハイパーパラメータ調整を一挙に習得し、ついでにベイズ最適化とSMBOと呼ばれる、ハイパーパラメータの調整の為に開発された変分法チックな手法のさわりも学ぶ。
個人的に変分法・感度解析系の最適化が好きです。(知らんがな)  
ベイス推定かベイズ最適化はどっかで使うと思います。
#### 今回は機械学習なのであんまり効果が実感できず、しょぼいかもしれませんが、深層学習だと重宝します。深層学習のパラメータはニューロン数、ドロップアウト率、バッチサイズやエポック数などなど設定するパラメータが死ぬほど多い・広いので重宝します。実際はハイパーパラメータの調整はGAじゃなくてベイズ最適化か、SMBOの方が早くていいです。GAは遅いけど、最適解を見つけやすいのがいいとこなので。

問題の設定。めんどくさいのでsklearnの糖尿病患者の診療データを使用します(回帰)  
目的変数 : 基準から1年後の疾患進行の定量的尺度  
説明変数 : 基本情報、低(高)密度リポタンパク質(コレステロール)、血清測定等

In [1]:
# import the date for machine learning
import random
import numpy as np
import pandas as pd
from deap import base, creator, tools, algorithms

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC, SVC
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.pipeline import Pipeline
from sklearn.metrics import r2_score
from sklearn.datasets import load_diabetes
from sklearn.model_selection import GridSearchCV

# データを見るだけ。
dataset = load_diabetes()

# set dataframe
X = pd.DataFrame(dataset.data, columns=dataset.feature_names)
y = pd.DataFrame(dataset.target, columns=['y'])

# check the shape
print('----------------------------------------------------------------------------------------')
print('X shape: (%i,%i)' %X.shape)
print('y shape: (%i,%i)' %y.shape)
print('----------------------------------------------------------------------------------------')
print(y.describe())
print('----------------------------------------------------------------------------------------')
print(X.join(y).head())

/usr/local/var/pyenv/versions/anaconda3-5.2.0/lib/python3.6/site-packages/deap/tools/_hypervolume/pyhv.py:33: ImportWarning: Falling back to the python version of hypervolume module. Expect this to be very slow.
  "module. Expect this to be very slow.", ImportWarning)
/usr/local/var/pyenv/versions/anaconda3-5.2.0/lib/python3.6/importlib/_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
/usr/local/var/pyenv/versions/anaconda3-5.2.0/lib/python3.6/importlib/_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)


----------------------------------------------------------------------------------------
X shape: (442,10)
y shape: (442,1)
----------------------------------------------------------------------------------------
                y
count  442.000000
mean   152.133484
std     77.093005
min     25.000000
25%     87.000000
50%    140.500000
75%    211.500000
max    346.000000
----------------------------------------------------------------------------------------
        age       sex       bmi        bp        s1        s2        s3  \
0  0.038076  0.050680  0.061696  0.021872 -0.044223 -0.034821 -0.043401   
1 -0.001882 -0.044642 -0.051474 -0.026328 -0.008449 -0.019163  0.074412   
2  0.085299  0.050680  0.044451 -0.005671 -0.045599 -0.034194 -0.032356   
3 -0.089063 -0.044642 -0.011595 -0.036656  0.012191  0.024991 -0.036038   
4  0.005383 -0.044642 -0.036385  0.021872  0.003935  0.015596  0.008142   

         s4        s5        s6      y  
0 -0.002592  0.019908 -0.017646  151.0  
1 -

例として早くてわかりやすいSVCでやります。  
まず、ハイパーパラメータ(引数)を調べます。

In [2]:
from inspect import signature
sig = signature(LinearSVC)
sig

<Signature (penalty='l2', loss='squared_hinge', dual=True, tol=0.0001, C=1.0, multi_class='ovr', fit_intercept=True, intercept_scaling=1, class_weight=None, verbose=0, random_state=None, max_iter=1000)>

In [3]:
sig.parameters

mappingproxy({'penalty': <Parameter "penalty='l2'">,
              'loss': <Parameter "loss='squared_hinge'">,
              'dual': <Parameter "dual=True">,
              'tol': <Parameter "tol=0.0001">,
              'C': <Parameter "C=1.0">,
              'multi_class': <Parameter "multi_class='ovr'">,
              'fit_intercept': <Parameter "fit_intercept=True">,
              'intercept_scaling': <Parameter "intercept_scaling=1">,
              'class_weight': <Parameter "class_weight=None">,
              'verbose': <Parameter "verbose=0">,
              'random_state': <Parameter "random_state=None">,
              'max_iter': <Parameter "max_iter=1000">})

まあ、意味はわかりずらいので、documentationをみた方がいいです。    
https://scikit-learn.org/stable/modules/generated/sklearn.svm.LinearSVC.html

調整するパラメータ | 説明
--- | ---
penalty |  罰則項。L1正則化・L2正則化(デフォルト)を選択可
loss | 評価関数。ヒンジ損失か二乗ヒンジ(デフォルト)
dual | 双対問題を解くか否か(デフォルトはtrue)
tol | アルゴリズムの終了条件(default=1e-4)
C | ソフトマージンの厳しさを表すパラメータ

どうやらpenalty, loss, dualの組み合わせが  
l2,hinge,True,   l2, squared_hinge, True  
l1,squared_hinge, False,  l2, squared_hinge, False  
しか許さないみたいですね。

In [4]:
# 評価関数をクラスで定義する。
# まあ、こうした方がやりやすいと思う。

class LSVC():
    def __init__(self,
                 penalty='l2',
                 loss='squared_hinge',
                 dual=True,
                 tol=1e-4,
                 C=1.0):
        self.penalty = penalty
        self.loss = loss
        self.dual = dual
        self.tol = tol
        self.C = C
        params = """
        penalty:\t{0}
        loss:\t{1}
        dual:\t{2}
        tol:\t{3}
        C:\t{4}
        """.format(self.penalty, self.loss,
                   self.dual, self.tol,
                   self.C)
        print(params)

    # load diabetes data from sklearn dataset
    def get_data(self):
        dataset = load_diabetes()
        # set dataframe
        X = pd.DataFrame(dataset.data, columns=dataset.feature_names)
        y = pd.DataFrame(dataset.target, columns=['y'])
        X_train_, X_test_, self.y_train, self.y_test = train_test_split(X, y, test_size=0.20)

        # standard scaling for modeling
        # めんどくさいのでX_trainとかを標準化したものにします。
        scl = StandardScaler()
        self.X_train = scl.fit_transform(X_train_)
        self.X_test = scl.transform(X_test_)

        return self.X_train, self.X_test, self.y_train, self.y_test


    def make_model(self):
        self.model = LinearSVC(penalty=self.penalty,loss=self.loss,
                         dual=self.dual, tol=self.tol, C=self.C)

        return self.model

    # fit model
    def model_fit(self):
        self.get_data()
        self.make_model()
        self.model.fit(self.X_train,self.y_train)

    # evaluate model
    def model_evaluate(self):
        try :
            self.model_fit()
            evaluation = float(r2_score(self.y_test.as_matrix().ravel(), self.model.predict(self.X_test)))
        except ValueError:
            evaluation = float(-100)
        return evaluation,
# まじでこのreturnのカンマを忘れる。
# 単目的最適化でweight=(1.0,)としているから必要
# function to run mnist class


def run_model(individual):

    _lsvc = LSVC(penalty=individual[0][0],
                 loss=individual[0][1],
                 dual=individual[0][2],
                 tol=individual[1],
                 C=individual[2])
    model_evaluation = _lsvc.model_evaluate()
    return model_evaluation


In [5]:
creator.create('FitnessMax', base.Fitness, weights =(1.0,) )
creator.create('Individual', list , fitness = creator.FitnessMax)

# defining attributes for individual
toolbox = base.Toolbox()

# hyper parameter
#toolbox.register("penalty", random.choice, ('l1', 'l2'))
#toolbox.register("loss", random.choice, ('hinge', 'squared_hinge'))
#toolbox.register("dual", random.choice, (True, False))
toolbox.register("penalty_loss_dual", 
                 random.choice, (('l2','hinge',True),('l2','squared_hinge',True),
                             ('l1','squared_hinge',False),('l2','squared_hinge',False)) )
toolbox.register("tol", random.uniform, 1e-2, 1e-6)
toolbox.register("C", random.uniform, 0, 1000)

# register attributes to individual
toolbox.register('individual', tools.initCycle, creator.Individual,
                 (toolbox.penalty_loss_dual, toolbox.tol,
                  toolbox.C),
                  n = 1)
# individual to population
toolbox.register('population', tools.initRepeat, list, toolbox.individual)

# 突然変異を定義
def mutChoice(individual, indpb):

    # ループを諦める。
    if random.random() < indpb:
        individual[0] = random.choice((('l2','hinge',True),('l2','squared_hinge',True),
                             ('l1','squared_hinge',False),('l2','squared_hinge',False)))
    if random.random() < indpb:
            individual[1] = random.uniform(1e-2, 1e-6)
    if random.random() < indpb:
            individual[2] = random.uniform(0, 1000)
    return individual,

# evolution
toolbox.register('mate', tools.cxTwoPoint)
toolbox.register('mutate', mutChoice, indpb = 0.05)
toolbox.register('select', tools.selTournament, tournsize=3)
toolbox.register('evaluate', run_model)


In [6]:
def main():
    random.seed(64)
       
    pop = toolbox.population(n=10)
    
    hof = tools.HallOfFame(1, similar=np.array_equal)
    
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", np.mean)
    stats.register("std", np.std)
    stats.register("min", np.min)
    stats.register("max", np.max)
    
    algorithms.eaSimple(pop, toolbox, cxpb=0.5, mutpb=0.2, ngen=10, stats=stats,halloffame=hof)

    return pop, stats, hof

In [7]:
pop, stats, hof = main()


        penalty:	l2
        loss:	squared_hinge
        dual:	False
        tol:	0.008751614920157491
        C:	612.989361613698
        


/usr/local/var/pyenv/versions/anaconda3-5.2.0/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/var/pyenv/versions/anaconda3-5.2.0/lib/python3.6/site-packages/ipykernel_launcher.py:60: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.



        penalty:	l2
        loss:	hinge
        dual:	True
        tol:	0.007965092174156621
        C:	268.64853280867965
        

        penalty:	l2
        loss:	squared_hinge
        dual:	True
        tol:	0.0030141239913079607
        C:	778.191675640411
        

        penalty:	l2
        loss:	squared_hinge
        dual:	True
        tol:	0.00920195917472651
        C:	416.7337673905216
        

        penalty:	l2
        loss:	squared_hinge
        dual:	True
        tol:	0.0027878352960267793
        C:	83.66591038506866
        

        penalty:	l2
        loss:	hinge
        dual:	True
        tol:	0.0007244597822858405
        C:	174.56355024436598
        

        penalty:	l2
        loss:	hinge
        dual:	True
        tol:	0.0005005909187553673
        C:	264.0057520512484
        

        penalty:	l2
        loss:	squared_hinge
        dual:	True
        tol:	0.009111872041927952
        C:	29.54361416160878
        

        penalty:	l2
        loss:	squar


        penalty:	l2
        loss:	squared_hinge
        dual:	True
        tol:	0.0027878352960267793
        C:	264.0057520512484
        
8  	5     	-0.102441 	0.0991709	-0.302227	0.0708499  

        penalty:	l2
        loss:	squared_hinge
        dual:	True
        tol:	0.0027878352960267793
        C:	778.191675640411
        

        penalty:	l2
        loss:	squared_hinge
        dual:	True
        tol:	0.0027878352960267793
        C:	778.191675640411
        

        penalty:	l2
        loss:	squared_hinge
        dual:	True
        tol:	0.0027878352960267793
        C:	264.0057520512484
        

        penalty:	l2
        loss:	squared_hinge
        dual:	True
        tol:	0.0027878352960267793
        C:	264.0057520512484
        

        penalty:	l2
        loss:	squared_hinge
        dual:	True
        tol:	0.0027878352960267793
        C:	778.191675640411
        

        penalty:	l2
        loss:	squared_hinge
        dual:	True
        tol:	0.0027878352960267793


In [8]:
hof.items

[[('l2', 'squared_hinge', True), 0.0027878352960267793, 778.191675640411]]

In [9]:
hof.keys

[deap.creator.FitnessMax((0.3082276885918732,))]

#### LinearSVCではほとんどゴミみたいな数値しか出なかったのに、そこそこのr2スコアで出てきます。一過性のものでないか、実験します。

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)
pipe_lsvc = Pipeline([('scl',StandardScaler()),
                      ('est',LinearSVC(penalty='l2',loss='squared_hinge', 
                                       dual=True, tol=0.0027878352960267793,
                                      C=778.191675640411))])
pipe_lsvc.fit(X_train, y_train)
r2_score(y_test.values, pipe_lsvc.predict(X_test))


/usr/local/var/pyenv/versions/anaconda3-5.2.0/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.0939021112595102

そもそも、LinearSVCではどう頑張ってもこのデータの回帰は不可能のようです。笑  
train_test_splitがドンピシャな時だけましなスコアが出ます。
全然いい例じゃなかったですね。  
train_test_splitのrandom_stateを固定したらいい値は出るでしょうが、それは微妙ですね。過学習になりそうです。  
あと、最後のpopの中に割といいやつもいると思います。

# 課題
## GradientBoostingでハイパーパラメータを選んで同じようにハイパーパラメータを調整してみましょう。

GradientBoostingなら、そこそこのスコアは出ます。  
そして、GradientBoostingは結構パラメータに影響されやすいけど、ドンピシャにハマると機械学習の中では最高の精度が出ます。  
パラメータ選択して、パラメータの数から、popsizeと、gaのループ回数とかをうまく計算時間と照らし合わせて決める感じですね。  